In [1]:
import numpy as np
import pandas as pd

### Загрузка данных

In [2]:
train_df = pd.read_csv('./classification_data/train.csv')
test_df = pd.read_csv('./classification_data/test.csv')

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   10000 non-null  int64  
 1   age                  10000 non-null  float64
 2   years_of_experience  10000 non-null  float64
 3   lesson_price         10000 non-null  float64
 4   qualification        10000 non-null  float64
 5   physics              10000 non-null  float64
 6   chemistry            10000 non-null  float64
 7   biology              10000 non-null  float64
 8   english              10000 non-null  float64
 9   geography            10000 non-null  float64
 10  history              10000 non-null  float64
 11  mean_exam_points     10000 non-null  float64
 12  choose               10000 non-null  int64  
dtypes: float64(11), int64(2)
memory usage: 1015.8 KB


In [4]:
train_df.head(5)

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
0,0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0
1,1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0,1
2,2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0
3,3,33.0,3.0,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,1
4,4,46.0,3.0,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0,0


Целевая переменная - 'choose'

Посмотрим на нее

In [5]:
train_df['choose'].value_counts()

0    8891
1    1109
Name: choose, dtype: int64

Как видно, классы несбалансированы. Это может доставить проблемы

Вынесем признаки, на которых будет обучаться модель в отдельный список

In [6]:
base_features = train_df.columns.drop(['Id', 'choose']).tolist()

In [7]:
train_df[base_features].describe()

,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000
mean,45.800900,1.974800,1702.440000,1.724300,0.37060,0.121500,0.117200,0.059100,0.02770,0.018000,64.435200
std,8.030274,1.766883,523.789062,0.798845,0.48299,0.326724,0.321675,0.235824,0.16412,0.132958,13.595024
min,23.000000,0.000000,200.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,33.000000
25%,40.000000,0.000000,1300.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,55.000000
50%,46.000000,2.000000,1550.000000,2.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,63.000000
75%,51.000000,3.000000,2150.000000,2.000000,1.00000,0.000000,0.000000,0.000000,0.00000,0.000000,74.000000
max,68.000000,9.000000,3950.000000,4.000000,1.00000,1.000000,1.000000,1.000000,1.00000,1.000000,100.000000


В качестве модели будем использовать Логистическую регрессию. Необходимо отмасштабировать ряд признаков

### Подготовка данных

Классы для масштабирования признаков

In [8]:
class Standard:
    '''StandardScaler'''
    def __init__(self):
        self.mean = None
        self.std = None
    
    def _std_scale(self, X, fit=False):
        if fit:
            self.mean, self.std = X.mean(axis=0), X.std(axis=0)
            
        mean, std = self.mean, self.std
        
        if mean is None or std is None:
            mean, std = X.mean(axis=0), X.std(axis=0)
            
        return (X - mean)/std
    
    def fit_transform(self, X):
        X = X.values
        X = self._std_scale(X, fit=True)
        
        return X
        
    def transform(self, X):
        X = X.values
        X = self._std_scale(X)
        
        return X    
    
    
class MinMax:
    '''MinMaxScaler'''
    def __init__(self):
        self.Min = None
        self.Max = None
    
    def _Min_Max_scale(self, X, fit=False):
        if fit:
            self.Min, self.Max = X.min(axis=0), X.max(axis=0)
            
        Min, Max = self.Min, self.Max
        
        if Min is None or Max is None:
            Min, Max = X.min(axis=0), X.max(axis=0)
            
        return (X - Min)/(Max - Min)
    
    def fit_transform(self, X):
        X = X.values
        X = self._Min_Max_scale(X, fit=True)
        
        return X
        
    def transform(self, X):
        X = X.values
        X = self._Min_Max_scale(X)
        
        return X    

Разбиение выборки на обучающую и валидационную подвыборки.

In [9]:
from sklearn.model_selection import train_test_split

X = train_df[base_features]
y = train_df['choose']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=100)

Масштабирование признаков. 

In [10]:
features_std = ['age', 'years_of_experience', 'lesson_price', 'qualification', 'mean_exam_points']

# стандартизация
scaler = Standard()

X_train_scaled = X_train.copy()
X_valid_scaled = X_valid.copy()

X_train_scaled[features_std] = scaler.fit_transform(X_train[features_std])
# валидационная выборка масштабируется относительно обучающей выборки
X_valid_scaled[features_std] = scaler.transform(X_valid[features_std])

In [11]:
X_train_scaled.head(5)

,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
8369,-0.851802,0.592699,-0.859098,-0.899539,0.0,0.0,0.0,0.0,0.0,0.0,0.335736
9722,0.269133,0.592699,-1.144877,-0.899539,0.0,0.0,0.0,0.0,0.0,0.0,-0.987143
6950,1.016424,-1.108626,-0.287541,-0.899539,0.0,0.0,1.0,0.0,0.0,0.0,-1.942555
1919,-2.470931,1.159807,1.427131,0.348304,1.0,0.0,0.0,0.0,0.0,0.0,0.262242
5713,1.514617,-0.541518,-0.478060,0.348304,0.0,0.0,0.0,0.0,0.0,0.0,-0.693170


## Балансировка классов

Для балансировки классов будет использоваться алгоритм SMOTE. 

$$ X' = X + rand(0, 1)*(X - X_k) $$

Где 

$ X $ - исходный (оригинальный) пример

$ X' $ - синтезированный пример

$ rand(0, 1) $ - случайное значение от 0 до 1

$ X_k $ - примеры из k-ближайших соседей (в том же классе)



Будем брать только одного (случайного) соседа из k-ближайших. Реализуем алгоритм в коде. 

In [12]:
from scipy.stats import mode


def self_knn(X, k):
    '''Функция возвращает k-ближайших объектов массива для каждого наблюдения в данном массиве.'''
    
    # Вычисление евклидового расстояния до каждого объекта в X
    distances = np.linalg.norm(X[:, np.newaxis] - X, axis=2, ord=2)
    
    # Нахождение индексов k-ближайших соседей
    # Нулевой индекс - сам объект. 
    k_nearest_idx = np.argpartition(distances, range(k))[:, :k]
    
    return X[k_nearest_idx]


def balancing(X, y, k=3):
    '''Балансировка классов путем сэмплирования. Собственная реализация алгоритма SMOTE.
    Генерирование синтетических данных, похожих на минорный класс'''
    
    # определение индексов категориальных (бинарных) признаков
    categorical = []
    for i in range(X.shape[1]):
        if len(np.unique(X[:, i])) < 10:
            categorical.append(i)
    
    # классы и количество наблюдений
    classes, counts = np.unique(y, return_counts=True)
    # определение мажорного и минорного классов
    major_class = classes[np.argmax(counts)]
    minor_class = classes[np.argmin(counts)]
    # разделение данных
    major_class_data = X[y == major_class]
    minor_class_data = X[y == minor_class]
    # количество наблюдений
    major_len = major_class_data.shape[0]
    minor_len = minor_class_data.shape[0]
    
    # цикл до тех пор, пока количество наблюдений минорного класса меньше количества наблюдений мажорного
    while minor_len < major_len:
        # исходные наблюдения, из которых будут генерироваться данные
        origins = minor_class_data.copy()
        # разница между мажорным и минорным классом
        residual = major_len - minor_len
        # если разница меньше количества наблюдений в минорном классе,
        # то берется случайная подвыборка из минорного равная разнице
        if residual < minor_len:
            origins = origins[np.random.permutation(minor_len)[:residual]]
        
        # количество исходных наблюдений
        n_rows = origins.shape[0]
        
        # k-ближайших соседей в классе для каждого исходного наблюдения
        k_nearest = self_knn(origins, k)

        # мода по категориальным (бинарным) признакам у k-ближайших
        cat_mode = mode(k_nearest[:, :, categorical], axis=1).mode[:, 0, :]
        
        # случайные индексы среди k-ближайших
        neighbor_idx = np.random.randint(k, size=n_rows)
        
        # к исходным добавляется произведение вектора случайных значений (от 0 до 1) 
        # на разницу между исходными и одним (случайным) из k-ближайших
        synthetic = origins + np.random.rand(n_rows, 1) * (origins - k_nearest[range(n_rows), neighbor_idx, :])
        # замена категориальных на моду по категориальным признакам у k-ближайших
        synthetic[:, categorical] = cat_mode
        
        # объединение синтезированных данных и исходных
        # на следующей итерации наблюдения будут отбираться так же и из сгенерированных
        minor_class_data = np.vstack([minor_class_data, synthetic])

        minor_len = minor_class_data.shape[0]
    
    # объединение данных мажорного и минорного классов
    balanced_data = np.vstack([minor_class_data, major_class_data])
    
    # объединение меток мажорного и минорного классов
    minor_labels, major_labels = np.ones(minor_len) * minor_class, np.ones(major_len) * major_class
    balanced_labels = np.hstack([minor_labels, major_labels]).astype('int')
    
    # перемешивание данных и меток
    shuffled_indices = np.random.permutation(balanced_labels.shape[0])

    return balanced_data[shuffled_indices], balanced_labels[shuffled_indices]

In [13]:
%%time
balanced_data, balanced_labels = balancing(X_train_scaled.values, y_train.values, k=3)

CPU times: user 1.95 s, sys: 1.31 s, total: 3.26 s
Wall time: 3.31 s


In [14]:
print('\nДо балансировки:\n')
for label, count in zip(*np.unique(y_train.values, return_counts=True)):
    print(f'Класс {label}, Количество: {count}')
    
print('\nПосле балансировки:\n')
for label, count in zip(*np.unique(balanced_labels, return_counts=True)):
    print(f'Класс {label}, Количество: {count}')


До балансировки:

Класс 0, Количество: 7130
Класс 1, Количество: 870

После балансировки:

Класс 0, Количество: 7130
Класс 1, Количество: 7130


Классы сбалансированы.

## Модели и метрики

Логистическая регрессия

In [15]:
class LogRegression:
    
    '''Логистическая регрессия.
    
    max_iter - (int) количество итераций, по умолчанию 1000.
    
    fit_intercept - (bool) по умолчанию True.
    
    eta - (float) скорость обучения, по умолчанию 1.0
    
    l1 - (float) коэффициент регуляризации l1, по умолчанию 0.0. 
    Если значение отличное от 0 - будет применена l1 регуляризация.
    Возможно сочетание с l2 регуляризацией (ElasticNet).
    
    l2 - (float) коэффициент регуляризации l1, по умолчанию 0.0.
    Если значение отличное от 0 - будет применена l2 регуляризация.
    Возможно сочетание с l1 регуляризацией (ElasticNet).
    '''
    
    def __init__(self, max_iter=1000, fit_intercept=True, eta=1.0, l1=0.0, l2=0.0):
        self.max_iter = max_iter
        self.fit_intercept = fit_intercept
        self.eta = eta
        self.l1 = l1
        self.l2 = l2
        
    def _sigmoid(self, z):
        return 1/(1 + np.exp(-z))
    
    def _calc_pred(self, X, threshold=0.5):
        y_pred = self._sigmoid(X @ self.coef_)
        y_pred = (y_pred > threshold).astype('int')
        return y_pred
    
    def fit(self, X, y):
        n = X.shape[0]
        
        if self.fit_intercept:
            X = np.hstack([np.ones((n, 1)), X])
        
        W = np.zeros(X.shape[1])
        
        for _ in range(self.max_iter):
            y_pred = self._sigmoid(X @ W)
            # градиент весов
            dQ = 2/n * X.T @ (y_pred - y)
            # градиент регуляризации
            dReg = self.l1 * np.sign(W) + self.l2 * W
            
            # оптимизация весов
            W -= self.eta * dQ + dReg
            
        self.coef_ = W
            
        return self
    
    def predict(self, X, threshold=0.5):
        if self.fit_intercept:
            X = np.hstack([np.ones((X.shape[0], 1)), X])
            
        y_pred = self._calc_pred(X, threshold=threshold)
        
        return y_pred

Подбор лучших параметров модели

In [16]:
import numpy as np
from itertools import product


class BestParamsCV:
    
    '''Самописный аналог GridSearchCV.
    
    При инициализации передаются следующие параметры:
    
    model_class - класс модели, для которой подбираются лучшие параметры.
    
    score_func - функция оценки качества предсказания модели.
    
    parameters - (dict) ключ - параметр, значение - значения в виде списка.
    
    cv - (int), количество фолдов, на которое будет делиться выборка при кроссвалидации, по умолчанию 5.
    
    '''
    
    def __init__(self, model_class, score_func, parameters, cv=5):
        self.model_class = model_class
        self.score_func = score_func
        self.param_keys = list(parameters.keys())
        self.param_values = list(parameters.values())
        self.cv = cv
        
    def _cross_validation(self, X, y, params):
        
        # количество элементов в выборке
        n = len(y)
        
        # перемешивание индексов
        shuffled = np.random.permutation(np.arange(n))
        
        # список для оценок
        scores = []
        
        # задаем индекс конечного элемента валидационного фолда
        end = 0
        
        # циклом идем по количеству фолдов в обратном порядке, 
        # при 5 фолдах:
        # первая итерация - для валидации берем первую 1/5 выборки
        # вторая итерация - для валидации берем первую 1/4 оставшейся выборки (не попадает предыдущая часть)
        # ...
        for i in range(self.cv, 0, -1):
            # индекс начального элемента приравниваем индексу конечного
            start = end
            
            # к индексу конечного элемента прибавляем длину фолда:
            # от общей длины выборки отнимаем индекс начального элемента (длина оставшейся выборки) и 
            # целочисленно делим на оставшееся количество фолдов (i), 
            # прибавляем остаток от деления (чтобы все объекты участвовали в обучении/валидации).
            end += (n - start)//i + (n - start)%i
            
            # валидационный фолд
            _valid = shuffled[start:end]
            # для обучения объединяем фолды до валидационного и после
            _train = np.append(shuffled[:start], shuffled[end:])
            
            # инициирование модели с заданными гиперпараметрами
            model = self.model_class(**params)
            
            # обучение на тренировочных фолдах
            model.fit(X[_train], y[_train])
            # в список добавляется оценка качества предсказания на валидационном фолде
            scores.append(self.score_func(y[_valid], model.predict(X[_valid])))

        return np.mean(scores)
        
    def _get_params(self, values):
        return dict(zip(self.param_keys, values))
        
    def fit(self, X, y):
        
        best_score = 0
        best_params = None
        
        # цикл по всем возможным комбинациям гиперпараметров
        for values in product(*self.param_values):
            # получение гиперпараметров для модели в виде словаря
            params = self._get_params(values)
            # оценка качества с использованием кроссвалидации
            score = self._cross_validation(X, y, params)
            
            if score > best_score:
                best_score, best_params = score, params
                
        self.best_score_ = best_score
        self.best_params_ = best_params
        
        return self

Метрики оценки качества классификации

In [17]:
def calc_accuracy(y, y_pred):
    if not isinstance(y, (np.ndarray, list)) or not isinstance(y_pred, (np.ndarray, list)):
        raise TypeError('input must be a numpy.ndarray or list')
    else:
        y, y_pred = np.array(y), np.array(y_pred)
        if y.shape != y_pred.shape:
            raise ValueError(f'shape mismatch: "y" has shape {y.shape} and "y_pred" has shape {y_pred.shape}')
        else:
            return sum(y == y_pred)/(y.shape[0] + 1e-16)
    

def TPR(y, y_pred):
    if not isinstance(y, (np.ndarray, list)) or not isinstance(y_pred, (np.ndarray, list)):
        raise TypeError('input must be a numpy.ndarray or list')
    else:
        y, y_pred = np.array(y), np.array(y_pred)
        if y.shape != y_pred.shape:
            raise ValueError(f'shape mismatch: "y" has shape {y.shape} and "y_pred" has shape {y_pred.shape}')
        else:
            TP = sum((y == 1) & (y_pred == 1))
            FN = sum((y == 1) & (y_pred == 0))
            return TP/(TP + FN + 1e-16)


def FPR(y, y_pred):
    if not isinstance(y, (np.ndarray, list)) or not isinstance(y_pred, (np.ndarray, list)):
        raise TypeError('input must be a numpy.ndarray or list')
    else:
        y, y_pred = np.array(y), np.array(y_pred)
        if y.shape != y_pred.shape:
            raise ValueError(f'shape mismatch: "y" has shape {y.shape} and "y_pred" has shape {y_pred.shape}')
        else:
            FP = sum((y == 0) & (y_pred == 1))
            TN = sum((y == 0) & (y_pred == 0))
            return FP/(FP + TN + 1e-16)

        
def calc_roc_auc(y, y_pred):
    return np.trapz([0, TPR(y, y_pred), 1], x=[0, FPR(y, y_pred), 1], dx=0.1)

## Обучение модели

Для начала попробуем обучить базовую модель на несбалансированных данных

In [18]:
model = LogRegression()

model.fit(X_train_scaled.values, y_train.values)

calc_roc_auc(y_valid.values, model.predict(X_valid_scaled.values))

0.6079645218697061

Найдем лучшие гиперпараметры для модели

In [19]:
%%time
parameters = {'max_iter': range(2000, 8001, 2000), 
              'eta': [0.1, 0.5, 1], 
              'l1': [0, 1e-4, 1e-3], 
              'l2': [0, 1e-4, 1e-3]}

# проверка на сбалансированных данных, 
# поэтому в качестве метрики будет использоваться accuracy
bestparams = BestParamsCV(LogRegression, calc_accuracy, parameters)

bestparams.fit(balanced_data, balanced_labels)

CPU times: user 54min 23s, sys: 1h 51min 38s, total: 2h 46min 1s
Wall time: 22min 38s


In [20]:
bestparams.best_params_

{'max_iter': 8000, 'eta': 0.5, 'l1': 0, 'l2': 0.001}

In [21]:
bestparams.best_score_

0.7572931276297334

Обучим модель с лучшими гиперпараметрами

In [22]:
model = LogRegression(**bestparams.best_params_)

model.fit(balanced_data, balanced_labels)

calc_roc_auc(y_valid.values, model.predict(X_valid_scaled.values))

0.7953342884772109

После балансировки метрика ROC_AUC на отложенной выборке существенно возросла.

### Обучение модели на всех данных

Масштабирование признаков

In [23]:
features_std = ['age', 'years_of_experience', 'lesson_price', 'mean_exam_points']

X_test = test_df[base_features]

scaler = Standard()

X_scaled = X.copy()
X_test_scaled = X_test.copy()

X_scaled[features_std] = scaler.fit_transform(X[features_std])
X_test_scaled[features_std] = scaler.transform(X_test[features_std])

Балансировка классов

In [24]:
%%time
balanced_data, balanced_labels = balancing(X_scaled.values, y.values, k=3)

CPU times: user 3.34 s, sys: 3.84 s, total: 7.19 s
Wall time: 14.5 s


Обучение модели с ранее подобранными гиперпараметрами

In [25]:
model = LogRegression(**bestparams.best_params_)

model.fit(balanced_data, balanced_labels)

Сохранение предсказанных меток для тестовой выборки

In [26]:
pred = pd.DataFrame(test_df['Id'])
pred['choose'] = model.predict(X_test_scaled)
pred.to_csv('final_predict.csv', index=False)